In [ ]:
from zipfile import ZipFile
from urllib.request import urlopen
import pandas as pd
import os

def download_url(url, save_path):
    with urlopen(url) as dl_file:
        with open(save_path, 'wb') as out_file:
            out_file.write(dl_file.read())

URL = 'https://datasets.d2.mpi-inf.mpg.de/MobileHCI2018/MPIIMobileAttention.zip'

download_url(URL, '../MPIIMobileAttention.zip')

print ('url downloaded :) \n')

# open and save the zip file onto computer
url = urlopen(URL)
output = open('./MPIIMobileAttention.zip', 'wb')    # note the flag:  "wb"  
print ('opened \n')      
output.write(url.read())
print ('written \n')  
output.close()
print ('closed \n')  

# read the zip file as a pandas dataframe
df = pd.read_pickle('MPIIMobileAttention.zip')    # zip files       
print ('read a pkl \n')  

df.to_pickle("./MPIIMobileAttention.pkl")
print ('made a pkl \n')  
# if keeping on disk the zip file is not wanted, then:
# os.remove(zipName)   # remove the copy of the zipfile on disk



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from zipfile import ZipFile
import pandas as pd
import numpy as np
import os
from urllib.request import urlopen, Request
import pickle


URL = 'https://datasets.d2.mpi-inf.mpg.de/MobileHCI2018/MPIIMobileAttention.zip'

r = Request(URL)
b2 = [z for z in URL.split('/') if '.zip' in z][0] #gets just the '.zip' part of the url

with open(b2, "wb") as target:
    target.write(urlopen(r).read()) # saves to file to disk

zipfile = ZipFile(b2,'r')

namelist = zipfile.namelist()
print(namelist)

"""
f = zipfile.extractall()

"""
list_of_dfs = []

for name in namelist:
    df = pd.read_pickle(name)
    list_of_dfs.append(df)

big_df = pd.concat(list_of_dfs)

print(list_of_dfs)

big_df.to_pickle("./MPIIMobileAttention.pkl")

In [ ]:
df = big_df.dropna(axis='columns')

df.dropna(axis=1, how='all')

df.loc[:, df.ne(0).any()]

df.fillna(df.mean())

phone_df = df.loc[:,(df.columns.str.startswith("phone_")) | (df.columns.str.startswith("app_")) |
                    (df.columns.str.startswith("touch_")) | (df.columns.str.startswith("gps_")) |
                    (df.columns.str.startswith("screen")) | (df.columns.str.startswith("disp")) |
                    (df.columns.str.startswith("whatsapp")) | (df.columns.str.startswith("temp")) |
                    (df.columns.str.startswith("distance_cam"))]
rgb_df1 = df.loc[:,(df.columns.str.startswith("objectclass_")) | (df.columns.str.startswith("objectness_")) |
                    (df.columns.str.startswith("saliency_")) | (df.columns.str.startswith("segmentationclass_"))]
rgb_df2 = df[['object_seg', 'sem_seg',  'gaze_xundist', 'gaze_yundist', 'corner1_xundist', 'corner1_yundist',
            'corner2_xundist', 'corner2_yundist', 'corner3_xundist', 'corner3_yundist',  'corner4_xundist',
             'corner4_yundist', 'corner1_xundistext', 'corner1_yundistext', 'corner2_xundistext',
             'corner2_yundistext','corner3_xundistext', 'corner3_yundistext', 'corner4_xundistext',
             'corner4_yundistext','corner1ext_x', 'corner1ext_y', 'corner2ext_x', 'corner2ext_y',
             'corner3ext_x', 'corner3ext_y','corner4ext_x', 'corner4ext_y', 'face_detections_world']]
rgb_df = pd.concat([rgb_df1, rgb_df2], axis=1)

headimu_df = df.loc[:,(df.columns.str.startswith("accelerometer_")) | (df.columns.str.startswith("gyro_")) |
                    (df.columns.str.startswith("mobilephone_in_scene_vid")) | (df.columns.str.startswith("corner1_x")) | 
                    (df.columns.str.startswith("corner1_y")) | (df.columns.str.startswith("corner2_y")) |
                    (df.columns.str.startswith("corner2_x")) | (df.columns.str.startswith("corner3_x")) |
                    (df.columns.str.startswith("corner3_y")) | (df.columns.str.startswith("corner4_x")) |
                    (df.columns.str.startswith("corner4_y")) ]

depth_df = df.loc[:,df.columns.str.startswith("depth_")]

gaze_df = df[['saliency', 'depth', 'objectness', 'pupil_x', 'pupil_y',
            'gaze_x', 'gaze_y', 'diameter', 'major', 'minor', 'angle',
            'fix_dispersions', 'fix_durations', 'fix_centroids_y',
            'fix_centroids_x', 'fix_centroidsext_x', 'fix_centroidsext_y']]

egocentric_df = pd.concat([headimu_df, pd.concat([rgb_df, depth_df], axis = 1)], axis = 1) # the egocentric sensors if we want to revisit as the paper did

In [ ]:
X = pd.concat([headimu_df, pd.concat([rgb_df, pd.concat([depth_df, pd.concat([phone_df, gaze_df], axis = 1)], axis = 1)], axis = 1)], axis = 1)
X = pd.get_dummies(X)
X= np.array(X).astype(np.float32)

y = np.array(df['gaze_on_screen'])

print('X: \n', X.dtype, X)
print('y: \n',y.dtype, y)

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25, random_state=0)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

from proximityMatrix import proximityMatrix

proxMat = proximityMatrix(clf, X_train, normalize=True)

print("proxMat: \n", proxMat)

In [ ]:
# just to get current path rn
import os
print(os.getcwd())

In [ ]:
# DUMP

"""

df = loadData(files[0:1])
print(files[0:1])

#features.head()

with zipfile.open(name) as myfile:
  df = pickle.load(myfile)


df = df.drop(columns=['trust_screen', 'environment', 'indoor_outdoor','trust_activity',
                       'user_name', 'message', 'app_description','stat_mobile',
                       'screenstatus', 'screenactivity', 'phone_screenonoff_description',
                       'sem_seg','subject_folder','block_folder', 'object_seg', 'question_type'])



#'evironment_1', 'evironment_2', 'evironment_3', 'evironment_4', 'evironment_5', 

#df = df.fillna(0)

#df = df.replace([np.inf, -np.inf], np.nan)

df = df.dropna(axis='columns')

col_names = df.columns
print(col_names)

#df = pd.get_dummies(df)

print(df.head())


f = open(zipfile.NameToInfo.keys(),'wb')
f.write(zipfile.open(zipfile.NameToInfo.keys()[0]).read())
f.close()

df = pd.read_pickle(zipfile.NameToInfo.keys()[0])


# print("df: \n", df)
# print(df.dtypes)

# print (df)


"""

""""
text_files = zf.infolist()


print ("Uncompressing and reading data... ")

for text_file in text_files:
    print(text_file.filename)
    df = pd.read_pickle(zf.open(text_file.filename))     # do df manipulations
    list_of_dfs.append(df)

big_df = pd.concat(list_of_dfs) # df = pd.read_pickle(b2, compression='zip')  #opens the saved zip file

big_df = pd.concat(list_of_dfs, ignore_index = True, sort=True)
os.remove(b2) #removes the zip file

big_df.to_pickle("./MPIIMobileAttention.pkl")

""""